# 🎧 Moodle to MP3 Converter

**Steps:**
1. Use the Tampermonkey script to extract links from Moodle
2. Paste links in the cell below
3. Run all cells
4. Download your MP3 files!

In [ ]:
# Step 1: Install/Update yt-dlp
!pip install -U yt-dlp -q
print("✅ yt-dlp ready!")

In [ ]:
# Step 2: Paste your extracted links here (one per line)
links = """
https://example.cloudfront.net/video1.mp4
https://example.cloudfront.net/video2.mp4
"""

# Save to file
with open('links.txt', 'w') as f:
    f.write('\n'.join([l.strip() for l in links.strip().split('\n') if l.strip()]))
print(f"✅ Saved {len([l for l in links.strip().split(chr(10)) if l.strip()])} links")

In [ ]:
# Step 3: Download and convert to MP3
!yt-dlp -x --audio-format mp3 --audio-quality 192K -o "%(title)s.%(ext)s" -a links.txt
print("\n✅ Conversion complete!")

In [ ]:
# Step 4: Zip and download all MP3 files
import glob
from google.colab import files
import zipfile

mp3_files = glob.glob('*.mp3')
print(f"Found {len(mp3_files)} MP3 files")

if mp3_files:
    with zipfile.ZipFile('moodle_audio.zip', 'w') as zipf:
        for mp3 in mp3_files:
            zipf.write(mp3)
    files.download('moodle_audio.zip')
    print("✅ Downloading zip...")
else:
    print("❌ No MP3 files found. Check the download step above.")